In [21]:
import keras
from keras import layers, Input, optimizers, models
from keras.applications.vgg16 import VGG16
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
img_size = (224, 224)
BATCH_SIZE = 1
NUM_EPOCHS = 100
DATAPATCH = '2019_endoscopy_image/'
classes = ['normal', 'abnormal']

In [22]:
train_datagen = ImageDataGenerator(rescale=1./255)
train_data = train_datagen.flow_from_directory(
    directory=DATAPATCH+'train', target_size=img_size, batch_size=BATCH_SIZE, class_mode='binary', classes=classes)
test_datagen = ImageDataGenerator(rescale=1./255)
test_data = test_datagen.flow_from_directory(
    directory=DATAPATCH+'test', target_size=img_size, batch_size=BATCH_SIZE, class_mode='binary', classes=classes)
input_tensor = Input(shape=(224, 224, 3), dtype='float32', name='input')

Found 2561 images belonging to 2 classes.
Found 450 images belonging to 2 classes.


In [23]:
pre_VGG = VGG16(weights='imagenet', include_top=False,
                input_shape=(224, 224, 3))
pre_VGG.trainable = False
pre_VGG.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [24]:
allModel = models.Sequential()
allModel.add(pre_VGG)
allModel.add(layers.Flatten())
allModel.add(layers.Dense(512, activation='relu'))
allModel.add(layers.Dense(256, activation='relu'))
allModel.add(layers.Dense(1, activation='sigmoid'))
allModel.compile(optimizer='rmsprop',
                 loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history = allModel.fit_generator(train_data, epochs=NUM_EPOCHS,
                                 validation_steps=test_data.samples,
                                 steps_per_epoch=train_data.samples,
                                 validation_data=test_data).history

Epoch 1/100
2561/2561 [==============================] - 72s 28ms/step - loss: 5.4968 - acc: 0.6552 - val_loss: 4.4284 - val_acc: 0.7222
Epoch 2/100
2561/2561 [==============================] - 72s 28ms/step - loss: 5.4967 - acc: 0.6552 - val_loss: 4.4284 - val_acc: 0.7222
Epoch 3/100
1378/2561 [===============>..............] - ETA: 30s - loss: 5.4838 - acc: 0.6560

In [ ]:
loss = history['loss']
val_loss = history['val_loss']
acc = history['acc']
val_acc = history['val_acc']
epochs = range(1, len(loss)+1)
epochs2 = range(1,len(val_loss)+1)
plt.plot(epochs, loss, 'r', label='train loss')
plt.plot(epochs, acc, 'g', label='train acc')
plt.plot(epochs2, val_acc, 'b', label='Val acc')
plt.plot(epochs2, val_loss, 'c', label='Val loss')
plt.xlabel('Epochs')
plt.ylabel('Loss/Acc')
plt.legend()
plt.show()

In [ ]:
plt.plot(epochs, loss, 'r', label='train loss')
plt.plot(epochs2, val_loss, 'c', label='Val loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()
plt.plot(epochs, acc, 'g', label='train acc')
plt.plot(epochs2, val_acc, 'b', label='Val acc')
plt.xlabel('Epochs')
plt.ylabel('Acc')
plt.legend()
plt.show()